In [61]:
import os
import plaid
import requests
import datetime
import json
import pandas as pd
%matplotlib inline

In [62]:
def pretty_print_response(response):
  print(json.dumps(response, indent=4, sort_keys=True))

In [63]:
PLAID_CLIENT_ID = os.getenv('PLAID_CLIENT_ID')
#print(PLAID_CLIENT_ID)
PLAID_SECRET = os.getenv('PLAID_SBX_SECRET_KEY')
PLAID_PUBLIC_KEY = os.getenv('PLAID_PUBLIC_KEY')
PLAID_ENV = os.getenv('PLAID_ENV', 'sandbox')
PLAID_PRODUCTS = os.getenv('PLAID_PRODUCTS', 'transactions')

# Plaid Access Token

In this section, you will use the plaid-python api to generate the correct authentication tokens to access data in the free developer Sandbox. This mimics how you might connect to your own account or a customer account, but due to privacy issues, this homework will only require connecting to and analyzing the fake data from the developer sandbox that Plaid provides. 

Complete the following steps to generate an access token:
1. Create a client to connect to plaid
2. Use the client to generate a public token and request the following items: 
['transactions', 'income', 'assets']
3. Exchange the public token for an access token
4. Test the access token by requesting and printing the available test accounts

### 1. Create a client to connect to plaid

In [64]:
INSTITUTION_ID = "ins_109508"

In [65]:
# YOUR CODE HERE
client = plaid.Client(client_id=PLAID_CLIENT_ID, secret=PLAID_SECRET, public_key=PLAID_PUBLIC_KEY, environment='sandbox')

### 2. Generate a public token

In [66]:
# YOUR CODE HERE
create_tkn_response = client.Sandbox.public_token.create(INSTITUTION_ID, ['transactions', 'income', 'assets'])

### 3. Exchange the public token for an access token

In [67]:
# YOUR CODE HERE
exchange_response = client.Item.public_token.exchange(create_tkn_response['public_token'])

access_token = exchange_response['access_token']

# Fetch Accounts


In [68]:
# Fetch Accounts
client.Accounts.get(access_token)

{'accounts': [{'account_id': 'jnlKqpvnp1c7GnJBWBg7UGA1MVZAp9h1AwmDE',
   'balances': {'available': 100,
    'current': 110,
    'iso_currency_code': 'USD',
    'limit': None,
    'unofficial_currency_code': None},
   'mask': '0000',
   'name': 'Plaid Checking',
   'official_name': 'Plaid Gold Standard 0% Interest Checking',
   'subtype': 'checking',
   'type': 'depository'},
  {'account_id': '7DEWMPoDPdiAlnWmJmxAUaBEwJmBDWHgpjye5',
   'balances': {'available': 200,
    'current': 210,
    'iso_currency_code': 'USD',
    'limit': None,
    'unofficial_currency_code': None},
   'mask': '1111',
   'name': 'Plaid Saving',
   'official_name': 'Plaid Silver Standard 0.1% Interest Saving',
   'subtype': 'savings',
   'type': 'depository'},
  {'account_id': 'eW58k6LW67hPrDKkEk9Phr5eG475BmFLkEBpd',
   'balances': {'available': None,
    'current': 1000,
    'iso_currency_code': 'USD',
    'limit': None,
    'unofficial_currency_code': None},
   'mask': '2222',
   'name': 'Plaid CD',
   'officia

# Account Transactions with Plaid

In this section, you will use the Plaid Python SDK to connect to the Developer Sandbox account and grab a list of transactions. You will need to complete the following steps:


1. Use the access token to fetch the transactions for the last 90 days
2. Print the categories for each transaction type
3. Create a new DataFrame using the following fields from the JSON transaction data: `date, name, amount, category`. (For categories with more than one label, just use the first category label in the list)
4. Convert the data types to the appropriate types (i.e. datetimeindex for the date and float for the amount)

### 1. Fetch the Transactions for the last 90 days

In [69]:
# YOUR CODE HERE
start_date = '{:%Y-%m-%d}'.format(datetime.datetime.now() + datetime.timedelta(-90))
end_date = '{:%Y-%m-%d}'.format(datetime.datetime.now())

### 2. Print the categories for each transaction

In [71]:
# YOUR CODE HERE
transaction_response = client.Transactions.get(access_token, start_date, end_date)
print(json.dumps(transaction_response['transactions'][:2], indent=4, sort_keys=True))
for transaction in transaction_response['transactions']:
    print(transaction['category'])

[
    {
        "account_id": "7DEWMPoDPdiAlnWmJmxAUaBEwJmBDWHgpjye5",
        "account_owner": null,
        "amount": 25,
        "authorized_date": null,
        "category": [
            "Payment",
            "Credit Card"
        ],
        "category_id": "16001000",
        "date": "2020-04-15",
        "iso_currency_code": "USD",
        "location": {
            "address": null,
            "city": null,
            "country": null,
            "lat": null,
            "lon": null,
            "postal_code": null,
            "region": null,
            "store_number": null
        },
        "name": "CREDIT CARD 3333 PAYMENT *//",
        "payment_channel": "other",
        "payment_meta": {
            "by_order_of": null,
            "payee": null,
            "payer": null,
            "payment_method": null,
            "payment_processor": null,
            "ppd_id": null,
            "reason": null,
            "reference_number": null
        },
        "pending": fals

### 3. Create a new DataFrame using the following fields from the JSON transaction data: date, name, amount, category. 

(For categories with more than one label, just use the first category label in the list)

In [73]:
# YOUR CODE HERE
transaction_df = pd.DataFrame(columns=['date', 'name', 'amount', 'category'])
transaction = transaction_response['transactions']
for index, transaction_response in enumerate (transaction):
    transaction_df.loc[index] = [transaction['date'], transaction['name'], transaction['amount'], transaction['category']]
transaction_df.head()        
        

TypeError: list indices must be integers or slices, not str

### 4. Convert the data types to the appropriate types 

(i.e. datetimeindex for the date and float for the amount)

In [77]:
# YOUR CODE HERE
transaction_df.dtypes

date        object
name        object
amount      object
category    object
dtype: object

In [91]:
transaction_df['amount'] = transaction_df['amount'].astype("float")
transaction_df['amount']
transaction_df['date'] = pd.datetime(transaction_df['date'], format=['%Y, %m, %d'])
transaction_df = transaction_df.set_index('date')
transaction_df.head()

TypeError: cannot convert the series to <class 'int'>

---

# Income Analysis with Plaid

In this section, you will use the Plaid Sandbox to complete the following:
1. Determine the previous year's gross income and print the results
2. Determine the current monthly income and print the results
3. Determine the projected yearly income and print the results

In [24]:
# YOUR CODE HERE
income_response = client.Income.get(access_token)
pretty_print_response(income_response)

{
    "income": {
        "income_streams": [
            {
                "confidence": 0.99,
                "days": 720,
                "monthly_income": 500,
                "name": "UNITED AIRLINES"
            }
        ],
        "last_year_income": 6000,
        "last_year_income_before_tax": 7285,
        "max_number_of_overlapping_income_streams": 1,
        "number_of_income_streams": 1,
        "projected_yearly_income": 6085,
        "projected_yearly_income_before_tax": 7389
    },
    "request_id": "H0Tw7PDAFAogfWx"
}


In [85]:
print(f"Last Year's income {income_response['income']['last_year_income_before_tax']}")

Last Year's income 7285


In [87]:
print(f"Current monthly income {income_response['income']['income_streams'][0]['monthly_income']}")

Current monthly income 500


In [88]:
print(f"Projected Year's income {income_response['income']['projected_yearly_income_before_tax']}")

Projected Year's income 7389


---

# Budget Analysis
In this section, you will use the transactions DataFrame to analyze the customer's budget

1. Calculate the total spending per category and print the results (Hint: groupby or count transactions per category)
2. Generate a bar chart with the number of transactions for each category 
3. Calculate the expenses per month
4. Plot the total expenses per month

### Calculate the expenses per category

In [75]:
# YOUR CODE HERE
total_spending_per_category = transaction_df.groupby('category').sum()["amount"]
total_spending_per_category

Series([], Name: amount, dtype: float64)

In [76]:
total_spending_per_category.plot.pie()

TypeError: Empty 'DataFrame': no numeric data to plot

In [45]:
total_spending_per_category.plot.bar()

TypeError: Empty 'DataFrame': no numeric data to plot

### Calculate the expenses per month

In [80]:
# YOUR CODE HERE
transaction_df.reset_index(inplace=True)
transaction_df['month'] = pd.DatetimeIndex(transaction_df['date']).month
transaction_df.head()

,index,date,name,amount,category,month


In [ ]:
expenses_per_month = 